# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [1]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)



## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("../pset3/combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string
pharma = str(doj[doj["id"] == "17-1204"]["contents"])
print(pharma)


4909    The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain. "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.  We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.  And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspira

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [4]:
## your code here to restrict to alpha
def preprocess_text(text):
    words = text.split()
    
    processed_words = [word for word in words if word.isalpha()]
    
    processed_text = ' '.join(processed_words)
    
    return processed_text

pro_pharma = preprocess_text(pharma)

print(pro_pharma)

The founder and majority owner of Insys Therapeutics was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough than Americans died of synthetic opioid overdoses last and millions are addicted to And yet some medical professionals would rather take advantage of the addicts than try to help said Attorney General Jeff Justice Department will not tolerate We will hold accountable anyone from street dealers to corporate executives who illegally contributes to this nationwide And under the leadership of President we are fully committed to defeating this threat to the American of a current member of the Board of Directors of was arrested this morning in Arizona and charged with RICO as well as other including conspiracy to commit mail and wire fraud and conspiracy to violate the the former Executive Chairman of the Board and CEO of will app

In [5]:
## your code here for part of speech tagging

tokens = word_tokenize(pro_pharma)
tokens_pos = pos_tag(tokens)

all_adjectives = [token[0] for token in tokens_pos if token[1] == "JJ" or 
                  token[1] == "JJR" or token[1] == "JJS"]

from collections import Counter

word_counts = Counter(all_adjectives)
word_counts_df = pd.DataFrame(list(word_counts.items()), 
                              columns = ["Word", "Count"])
word_counts_df = word_counts_df.sort_values(by = "Count", ascending = False)
word_counts_5 = word_counts_df.head(n = 5)
print(word_counts_5)


          Word  Count
9       former      8
0   nationwide      4
8        other      3
28   addictive      3
25      opioid      3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [6]:
## your code here for part A
spacy_pharma = nlp(pharma) 

In [7]:
## your code here for part B
for token in spacy_pharma.ents:
    if token.label_ == "LAW":
        print("Entity: " + token.text + "; NER tag: " + token.label_)
        

Entity: RICO; NER tag: LAW
Entity: the Controlled Substances Act; NER tag: LAW
Entity: RICO; NER tag: LAW


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

In [8]:
## your code here 
print("""The Racketeer Influenced and Corrupt Organizations Act 
    was enacted in 1970 to strengthen legal tools for investigators in cases 
    involving organized crime, and after reading about 
    a pharmaceutical case involving RICO, it's because these companies are 
    functioning basically as organized crime, offering bribes 
    to have doctors perscribe their drug and disguising payments as speaking events.""")
      

The Racketeer Influenced and Corrupt Organizations Act 
    was enacted in 1970 to strengthen legal tools for investigators in cases 
    involving organized crime, and after reading about 
    a pharmaceutical case involving RICO, it's because these companies are 
    functioning basically as organized crime, offering bribes 
    to have doctors perscribe their drug and disguising payments as speaking events.


D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [9]:
## your code here
for token in spacy_pharma.ents:
    if (token.label_ == "DATE"): 
        if (re.search(r"year(?:s)?", token.text)):
            print(token.text)
            
        

last year
20 years
three years
five years
three years


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [10]:
## your code here
pattern = r"[A-Z][^.]*\b(?:year|years)\b(?:\s+\w+){0,6}"
matches = re.finditer(pattern, pharma)

for match in matches:
    print(match.group().strip())

print("""The CEO may be facing up to 20 years of imprisonment and 3 years 
    of supervised release if convicted.""")
    

More than 20,000 Americans died of synthetic opioid overdoses last year
The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine
The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a
The CEO may be facing up to 20 years of imprisonment and 3 years 
    of supervised release if convicted.


## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [15]:
## your code here for subsetting
doj_subset = doj[doj["topics_clean"].isin(["Civil Rights", "Hate Crimes", 
                                           "Project Safe Childhood"])]


B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [16]:
## your code here to define function
def function(text): 
    doc = nlp(text)
    clean_text = ' '.join([token.text for token in doc if not token.ent_type_])

    sentiment_scores = SentimentIntensityAnalyzer().polarity_scores(clean_text)
    return sentiment_scores
    


In [17]:
## your code here executing the function
text_scores = [function(text) for text in doj_subset["contents"]]

[{'neg': 0.18, 'neu': 0.773, 'pos': 0.047, 'compound': -0.992}, {'neg': 0.111, 'neu': 0.823, 'pos': 0.066, 'compound': -0.891}, {'neg': 0.091, 'neu': 0.838, 'pos': 0.07, 'compound': -0.8316}, {'neg': 0.115, 'neu': 0.807, 'pos': 0.077, 'compound': -0.9037}, {'neg': 0.167, 'neu': 0.781, 'pos': 0.052, 'compound': -0.9839}, {'neg': 0.144, 'neu': 0.809, 'pos': 0.047, 'compound': -0.9905}, {'neg': 0.139, 'neu': 0.779, 'pos': 0.082, 'compound': -0.9403}, {'neg': 0.087, 'neu': 0.852, 'pos': 0.061, 'compound': -0.7783}, {'neg': 0.121, 'neu': 0.824, 'pos': 0.055, 'compound': -0.9648}, {'neg': 0.148, 'neu': 0.797, 'pos': 0.054, 'compound': -0.9741}, {'neg': 0.225, 'neu': 0.743, 'pos': 0.032, 'compound': -0.9982}, {'neg': 0.09, 'neu': 0.849, 'pos': 0.06, 'compound': -0.8519}, {'neg': 0.077, 'neu': 0.862, 'pos': 0.061, 'compound': -0.6486}, {'neg': 0.332, 'neu': 0.638, 'pos': 0.03, 'compound': -0.9959}, {'neg': 0.179, 'neu': 0.761, 'pos': 0.06, 'compound': -0.9924}, {'neg': 0.116, 'neu': 0.818, 'po

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [18]:
#text_scores = [function(text) for text in doj_subset["contents"]]
doj_subset['sentiment_scores'] = doj_subset['contents'].apply(function)

In [19]:
## your code here
doj_subset_wscore = doj_subset.join(pd.json_normalize(doj_subset['sentiment_scores']))

# Sort the DataFrame by the negative score in descending order
doj_subset_wscore_sorted = doj_subset_wscore.sort_values(by='neg', ascending=False)

# Print the id, contents, and neg columns of the two most negative press releases
print(doj_subset_wscore_sorted[['id', 'contents', 'neg']].head(2))

          id  \
568   17-379   
513  11-1275   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [20]:
## agg and find the mean compound score by topic
mean_compound_scores = doj_subset_wscore.groupby('topics_clean')['compound'].agg('mean')

print(mean_compound_scores)



topics_clean
Civil Rights             -0.374800
Hate Crimes              -0.770914
Project Safe Childhood   -0.260291
Name: compound, dtype: float64


In [162]:
print("""The variation of these results reflects that the DOJ writes more negatively
    about hate crimes than the other two chosen topics, 
    which could be explained by the 
    inherent negativity of hate crimes""")


The variation of these results reflects that the DOJ writes more negatively
    about hate crimes than the other two chosen topics, 
    which could be explained by the 
    inherent negativity of hate crimes


# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [21]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [22]:
## your code defining a text processing function
# A. Write a function that:

# Takes in a single raw string in the contents column from that dataframe

# Does the following preprocessing steps:

# Converts the words to lowercase
# Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
# Only retains alpha words (so removes digits and punctuation)
# Only retains words 4 characters or longer
# Uses the snowball stemmer from nltk to stem
# Returns a joined preprocessed string

def preprocess(text):
    text = text.lower()
    
    custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]
    stop_words = set(stopwords.words('english') + custom_doj_stopwords)
    

    tokens = nltk.word_tokenize(text)

    tokens = [word for word in tokens if word.isalpha() and word not in stop_words and len(word) >= 4]
    

    stemmer = SnowballStemmer('english')
    
    stemmed = [stemmer.stem(word) for word in tokens]
    
    return ' '.join(stemmed)
    

In [23]:
## your code executing the function
doj_subset_wscore["processed_text"] = doj_subset_wscore["contents"].apply(preprocess) 
    

In [24]:
## your code showing the examples

specific_ids = ['16-718', '16-217']
filtered_data = doj_subset_wscore[doj_subset_wscore['id'].isin(specific_ids)]

print(filtered_data[['id', 'contents', 'processed_text']])

           id  \
6727   16-217   
11593  16-718   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [25]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [26]:
# your code here
matrix = create_dtm(list_of_strings = doj_subset_wscore["processed_text"], 
                    metadata = doj_subset_wscore[["id", "compound", "topics_clean"]])
print(matrix.head())


import re
from collections import Counter

def get_top_words(data, column, n=10):

    word_counts = Counter()
    for text in data[column]:
        words = re.findall(r'\b\w+\b', text.lower())  # Tokenize text
        word_counts.update(words)
 
    tf = pd.Series(word_counts).sort_values(ascending=False)
    
    return tf.head(n)


top_5_percent = doj_subset_wscore['compound'].quantile(0.95)
top_5_percent_data = doj_subset_wscore[doj_subset_wscore['compound'] >= top_5_percent]
print("Top 10 words for press releases with top 5% compound sentiment:")
print(get_top_words(top_5_percent_data, 'processed_text'))

bottom_5_percent = doj_subset_wscore['compound'].quantile(0.05)
bottom_5_percent_data = doj_subset_wscore[doj_subset_wscore['compound'] <= bottom_5_percent]
print("\nTop 10 words for press releases with bottom 5% compound sentiment:")
print(get_top_words(bottom_5_percent_data, 'processed_text'))

for topic in doj_subset_wscore['topics_clean'].unique():
    topic_data = doj_subset_wscore[doj_subset_wscore['topics_clean'] == topic]
    print(f"\nTop 10 words for press releases in topic '{topic}':")
    print(get_top_words(topic_data, 'processed_text'))
    

   index       id  compound            topics_clean  aaron  abandon  abbat  \
0     77  17-1235    0.8360            Civil Rights      0        0      0   
1    155  15-1522   -0.9444  Project Safe Childhood      0        0      0   
2    157   16-213   -0.9898  Project Safe Childhood      0        0      0   
3    162   16-381   -0.9905  Project Safe Childhood      0        0      0   
4    168   14-464   -0.8020             Hate Crimes      0        0      0   

   abbi  abbott  abdomen  ...  zamora  zane  zealand  zealous  zeeman  zero  \
0     0       0        0  ...       0     0        0        0       0     0   
1     0       0        0  ...       0     0        0        0       0     0   
2     0       0        0  ...       0     0        0        0       0     0   
3     0       0        0  ...       0     0        0        0       0     0   
4     0       0        0  ...       0     0        0        0       0     0   

   zobel  zone  zunggeemog  zwengel  
0      0     0    

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [ ]:
# your code here


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [ ]:
## your code here to get doc-level topic probabilities 

In [ ]:
## your code here to add those topic probabilities to the dataframe

In [ ]:
## your code here to summarize the topic proportions for each of the topics_clean 

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [192]:
## your code here 
## your code here 
def create_bigram_onedoc(text):
    # Split the text into words
    words = text.split()
    # Use zip to create pairs of consecutive words
    bigrams = [f"{w1}_{w2}" for w1, w2 in zip(words[:-1], words[1:])]
    # Join the bigrams into a single string separated by spaces
    return ' '.join(bigrams)

# Apply the function to the 'processed_text' column to create 'processed_text_bigrams'
doj_subset_wscore['processed_text_bigrams'] = doj_subset_wscore['processed_text'].apply(create_bigram_onedoc)

specific_press_release = doj_subset_wscore[doj_subset_wscore['id'] == '16-217']
print(specific_press_release[['id', 'processed_text', 'processed_text_bigrams']])

          id  \
6727  16-217   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [194]:
# your code here
def create_dtm_bigrams(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase=True, ngram_range=(2,2))
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    
    return dtm_dense_named_withid

# Select metadata columns and bigram text from the DataFrame
metadata_columns = doj_subset_wscore[['id', 'topics_clean', 'compound']]
bigram_text = doj_subset_wscore['processed_text_bigrams']

# Create the bigram document-term matrix
dtm_bigram = create_dtm_bigrams(bigram_text, metadata_columns)

# Check the output
print(dtm_bigram.head())

print("Dimensions of the unigram DTM:", matrix.shape)
print("Dimensions of the bigram DTM:", dtm_bigram.shape)
print("The bigram matrix has more dimensions than the unigram matrix primarily because bigrams capture more unique combinations of consecutive words, resulting in a higher number of columns because each column corresponds to a pair of words, and the combination of words into pairs produces a larger set of unique tokens than the set of unique single words.")


if 'processed_text_bigrams' in doj_subset_wscore.columns:
    # Print top bigrams for each topic
    topics = doj_subset_wscore['topics_clean'].unique()
    for topic in topics:
        topic_data = doj_subset_wscore[doj_subset_wscore['topics_clean'] == topic]
        top_bigrams = get_top_words(topic_data, 'processed_text_bigrams', 10)
        print(f"Top 10 bigrams for topic '{topic}':")
        print(top_bigrams)
else:
    print("processed_text_bigrams column not found. Please create it first.")

   index       id            topics_clean  compound  aaron_ford ford_memphi  \
0     77  17-1235            Civil Rights    0.8360                       0   
1    155  15-1522  Project Safe Childhood   -0.9444                       0   
2    157   16-213  Project Safe Childhood   -0.9898                       0   
3    162   16-381  Project Safe Childhood   -0.9905                       0   
4    168   14-464             Hate Crimes   -0.8020                       0   

   aaron_latham latham_munci  aaron_mcgrath mcgrath_websit  \
0                          0                             0   
1                          0                             0   
2                          0                             0   
3                          0                             0   
4                          0                             0   

   aaron_parrish parrish_arrest  aaron_parrish parrish_constitut  \
0                             0                                0   
1               

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 